In [1]:
# Import libraries
import seaborn as sb  
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_predict, cross_val_score, RepeatedKFold, cross_validate, GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.utils import resample 
from imaging_features import hyperintensities_flair, dti, cbf, resting_state_activation, ventricular_volume, surface_area, cortical_thickness

### Helper functions
The following helper functions are used to: 
- `train_model`: select model and scaler type and train the model
- `predict_ages`: apply the model to the test sample to obtain the predicted ages
- `evaluate_feature_ablation`: ablate one out of seven sets of feautres iteratively, and obtain the results for test sample(s) for both patients and health controls (optionally)
- `calculate_mae`: evaluate the performance of the model using mean absolute error (MAE)
- `plot_scatter_with_regression`: plot the scatter plot of the predicted age vs the chronological age


In [2]:
def train_model(X_train, y_train, model_type='ridge', scaler_type='minmax'):
    """
    Train a regression model (Ridge or SVM) with cross-validation to find optimal parameters.
    
    Args:
        X_train: Training features
        y_train: Training labels (ages)
        model_type: Type of model ('ridge' or 'svm')
        scaler_type: Type of scaler ('minmax' or 'standard')
    
    Returns:
        tuple containing:
        - trained_model: Optimized regression model
        - scaler: Fitted scaler (MinMaxScaler or StandardScaler)
    """
    # Choose scaler based on model type
    scaler = MinMaxScaler() if scaler_type == 'minmax' else StandardScaler()
    X_train_normalized = scaler.fit_transform(X_train)
    
    # Setup cross-validation
    cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
    
    if model_type == 'ridge':
        # Ridge Regression with cross-validation
        alphas = 10**np.linspace(10, -2, 100) * 0.5
        ridgecv = RidgeCV(alphas=alphas, scoring='neg_mean_absolute_error', cv=cv)
        ridgecv.fit(X_train_normalized, y_train)

        # Train final Ridge model with best alpha
        final_model = Ridge(alpha=ridgecv.alpha_)
    
    elif model_type == 'svm':
        # Support Vector Regression (SVR) with hyperparameter tuning
        param_grid = {
            'C': [0.1, 1, 10, 100],  # Regularization parameter
            'epsilon': [0.01, 0.1, 1],  # Tolerance margin for predictions
            'kernel': ['linear', 'rbf']  # Different kernels for flexibility
        }
        svr = SVR()
        grid_search = GridSearchCV(svr, param_grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        grid_search.fit(X_train_normalized, y_train)

        # Train final SVR model with best parameters
        final_model = SVR(**grid_search.best_params_)

    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    # Train the chosen model
    final_model.fit(X_train_normalized, y_train)
    
    return final_model, scaler


def predict_ages(model, scaler, X):
    """
    Make age predictions using trained model.
    
    Args:
        model: Trained Ridge regression model
        scaler: Fitted MinMaxScaler
        X: Features to predict on
        
    Returns:
        numpy array of predicted ages, rounded to 3 decimals
    """
    X_normalized = scaler.transform(X)
    predictions = model.predict(X_normalized)
    return np.round(predictions, 3)

def evaluate_feature_ablation(df_train_control, df_test_disease,  model_type, scalar_type,  df_test_control=None):
    """
      Train a regression model (Ridge or SVM) with cross-validation to find optimal parameters.

    This function normalizes the training features using the specified scaler, 
    then trains the selected model type using cross-validation to determine the best hyperparameters. 
    The trained model and the scaler used for normalization are returned for future predictions.

    Args:
        X_train (array-like): Training features (input data).
        y_train (array-like): Training labels (target values, e.g., ages).
        model_type (str, optional): Type of model to train ('ridge' for Ridge Regression or 'svm' for Support Vector Regression). Default is 'ridge'.
        scaler_type (str, optional): Type of scaler to use for normalization ('minmax' for MinMaxScaler or 'standard' for StandardScaler). Default is 'minmax'.

    Returns:
        tuple: A tuple containing:
            - trained_model: The optimized regression model (Ridge or SVR).
            - scaler: The fitted scaler (MinMaxScaler or StandardScaler) used for normalizing the input features.
    """
    results_disease = []
    results_control = []
    
    for feature in [hyperintensities_flair, dti, cbf, resting_state_activation, ventricular_volume, surface_area, cortical_thickness]:
        X_train_control = df_train_control.drop(['Age at Visit', 'BD#', 'Dx'] + feature, axis=1).astype('float64')
        X_test_disease = df_test_disease.drop(['Age at Visit', 'BD#', 'Dx'] + feature, axis=1).astype('float64')
        y_train_control =  df_train_control['Age at Visit'] # Train model
        
        model, scaler = train_model(X_train_control, y_train_control, model_type=model_type, scaler_type=scalar_type)
        y_pred_disease = predict_ages(model, scaler, X_test_disease)
        results_disease.append(y_pred_disease)

        if df_test_control is not None:
            X_test_control = df_test_control.drop(['Age at Visit', 'BD#', 'Dx'] + feature, axis=1).astype('float64')
            y_pred_control = predict_ages(model, scaler, X_test_control)
            results_control.append(y_pred_control)

    return results_disease, results_control


def calculate_mae(predictions_list, y_test):
    """
    Calculate the Mean Absolute Error (MAE) for a list of prediction arrays.

    Args:
        predictions_list: List of numpy arrays containing predicted values.
        y_test: Numpy array of ground truth values.

    Returns:
        List of MAE values for each prediction array.
    """
    mae_results = []
    for predictions in predictions_list:
        mae = np.mean(np.abs(predictions - y_test))
        mae_results.append(mae)
    return mae_results


def plot_scatter_with_regression(y_test, y_pred, title="", save_path=None):
    """Create a scatter plot of actual vs predicted ages with a regression line.
    
    Args:
        y_test (array-like): True age values.
        y_pred (array-like): Predicted age values.
        title (str, optional): Title of the plot.
        save_path (str, optional): Path to save the figure. If None, the plot is displayed instead.
    """
    plt.figure(figsize=(6, 6))  # Set figure size
    
    # Scatter plot
    plt.scatter(y_test, y_pred, alpha=0.7, edgecolors='k', linewidth=1)  

    # Regression line
    coefficients = np.polyfit(y_test, y_pred, 1)
    x_range = np.linspace(min(y_test), max(y_test), 100)
    y_range = np.polyval(coefficients, x_range)
    plt.plot(x_range, y_range, color='red', linewidth=2, label=f'Regression line: y={coefficients[0]:.2f}x + {coefficients[1]:.2f}')
    
    # Pearson correlation coefficient
    r, p_value = stats.pearsonr(y_test, y_pred)
    plt.text(0.05, 0.9, f'Pearson r: {r:.2f}', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

    # Grid, labels, and title
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.xlabel('Chronological Age', fontsize=12)
    plt.ylabel('Predicted Age', fontsize=12)
    plt.title(title if title else 'Scatter Plot with Regression Line', fontsize=14)
    plt.legend()

    # Save or show plot
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure directory exists
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()  # Close the figure to free memory
    else:
        plt.show()




In [3]:
feature_sets = [
    "hyperintensities_flair",
    "dti",
    "cbf",
    "resting_state_activation",
    "ventricular_volume",
    "surface_area",
    "cortical_thickness"]

Import datasets

In [4]:
df_control = pd.read_excel('data_original/reducedata_hc1011.xlsx', header=1)
df_disease = pd.read_excel('data_original/reducedata_bd1011.xlsx', header=1)

df_control = df_control.drop(df_control.columns[0:2], axis=1) # drop first two columns that show indices
df_disease = df_disease.drop(df_disease.columns[0:2], axis=1) # drop first two columns that show indices

complete_participants = pd.read_csv('data_original/subject_id_complete.csv')
complete_participants = complete_participants.rename(columns={"stdyptid": "BD#"}) #73 HC, 44 BD
df_control = df_control.merge(complete_participants, on=['BD#', 'Dx'], how='inner') # inner join drops the values that are not matched in the other df
df_disease = df_disease.merge(complete_participants,on=['BD#', 'Dx'], how='inner') # inner join drops the values that are not matched in the other df


### Training on full dataset of controls (73), Testing on full dataset of patients (44)

#### Ridge Regression

In [5]:
results_ridge_disease , _ = evaluate_feature_ablation(df_control, df_disease, 'ridge', 'minmax') # Predictions on BD sample from training on all HC sample


##### SVM

In [6]:
results_svm_disease , _ = evaluate_feature_ablation(df_control, df_disease, 'svm', 'minmax') # Predictions on BD sample from training on all 

##### Ridge Regression Results: MAE when tested on full dataset of patients (44)

In [7]:
y_test_disease =  df_disease['Age at Visit']
y_train_control =  df_control['Age at Visit']

# Example usage
mae_results_ridge_diease = calculate_mae(results_ridge_disease, y_test_disease)
print("MAE when the following feature set is ablated/omitted:")
for f, mae in zip(feature_sets,  mae_results_ridge_diease):
    print(f"{f}: {mae:.3f}")

MAE when the following feature set is ablated/omitted:
hyperintensities_flair: 10.579
dti: 10.513
cbf: 9.952
resting_state_activation: 10.134
ventricular_volume: 10.010
surface_area: 10.206
cortical_thickness: 9.743


##### SVM Results: MAE when tested on full dataset of patients (44)

In [8]:
# Example usage
mae_results_svm_diease = calculate_mae(results_svm_disease, y_test_disease)
print("MAE when the following feature set is ablated/omitted:")
for f, mae in zip(feature_sets,  mae_results_svm_diease):
    print(f"{f}: {mae:.3f}")

MAE when the following feature set is ablated/omitted:
hyperintensities_flair: 10.656
dti: 11.241
cbf: 10.350
resting_state_activation: 10.269
ventricular_volume: 10.349
surface_area: 10.661
cortical_thickness: 9.656


### Training on subset of age-matched controls (51), Test on full dataset of patients (44) and subset of controls (22)

In [9]:
subset_control_train = pd.read_csv('data_processed/hc_train_age_proportion_match_bd.csv')
subset_control_test = pd.read_csv('data_processed/hc_test_age_proportion_match_bd.csv')
subset_control_train = subset_control_train.drop(columns=['age_group'])
subset_control_test = subset_control_test.drop(columns=['age_group'])
y_train_control =  subset_control_train['Age at Visit']
y_test_control =  subset_control_test['Age at Visit']

##### Ridge Regression

In [10]:
results_ridge_disease_trained_on_subset, results_ridge_control_trained_on_subset = evaluate_feature_ablation(subset_control_train, df_disease, model_type='ridge', scalar_type='minmax',df_test_control = subset_control_test)

##### Ridge Regression Results: MAE when tested on full dataset of patients (44)

In [11]:
mae = calculate_mae(results_ridge_disease_trained_on_subset, y_test_disease)
print("MAE for each omitted feature set:")
for f, mae in zip(feature_sets, mae):
    print(f"{f}: {mae:.3f}")

MAE for each omitted feature set:
hyperintensities_flair: 10.949
dti: 10.960
cbf: 10.410
resting_state_activation: 10.579
ventricular_volume: 10.346
surface_area: 10.587
cortical_thickness: 9.932


##### Ridge Regression Results: MAE when tested on subset of controls (22)

In [12]:
mae_ridge_control_trained_on_subset= calculate_mae(results_ridge_control_trained_on_subset, y_test_control)
print("MAE for each omitted feature set:")
for f, mae in zip(feature_sets, mae_ridge_control_trained_on_subset):
    print(f"{f}: {mae:.3f}")

MAE for each omitted feature set:
hyperintensities_flair: 11.194
dti: 11.301
cbf: 11.154
resting_state_activation: 10.519
ventricular_volume: 11.838
surface_area: 10.853
cortical_thickness: 11.920


In [13]:
results_svm_disease_trained_on_subset, results_svm_control_trained_on_subset = evaluate_feature_ablation(subset_control_train, df_disease, model_type='svm', scalar_type='minmax',df_test_control = subset_control_test)

##### SVM Results: MAE when tested on full dataset of patients (44)

In [14]:
mae = calculate_mae(results_svm_disease_trained_on_subset, y_test_disease)
print("MAE for each omitted feature set:")
for f, mae in zip(feature_sets, mae):
    print(f"Omit {f}: {mae:.3f}")

MAE for each omitted feature set:
Omit hyperintensities_flair: 11.654
Omit dti: 11.889
Omit cbf: 11.401
Omit resting_state_activation: 11.326
Omit ventricular_volume: 10.929
Omit surface_area: 11.402
Omit cortical_thickness: 10.428


##### SVM Results: MAE when tested on subset of controls (22)

In [15]:
mae = calculate_mae(results_svm_control_trained_on_subset, y_test_control)
print("MAE for each omitted feature set:")
for f, mae in zip(feature_sets, mae):
    print(f"Omit {f}: {mae:.3f}")

MAE for each omitted feature set:
Omit hyperintensities_flair: 11.989
Omit dti: 11.719
Omit cbf: 11.211
Omit resting_state_activation: 11.190
Omit ventricular_volume: 12.255
Omit surface_area: 11.352
Omit cortical_thickness: 12.605


### Plot scatter plots of chronological age vs predicted age for each feature set
- Plots below uses the results from Ridge Regression trained on subset of controls (51) and tested on subset of controls (22)


In [21]:
save_folder = "figures/multimodal_model"
os.makedirs(save_folder, exist_ok=True)

for f, x in zip(feature_sets, results_ridge_control_trained_on_subset):
    title = f'Feature ablated: {f}'
    save_path = os.path.join(save_folder, f"results_ridge_control_trained_on_subset_{f}.png")
    plot_scatter_with_regression(y_test_control, x, title=title, save_path=save_path)


In [22]:
for f, x in zip(feature_sets, results_ridge_disease_trained_on_subset):
    title = f'Feature ablated: {f}'
    save_path = os.path.join(save_folder, f"results_ridge_disease_trained_on_subset_{f}.png")
    plot_scatter_with_regression(y_test_disease, x, title=title, save_path=save_path)
